# Aviation Venture Risk EDA

## Introduction

ACME Co. is interested in purchasing and operating airplanes for commercial and private enterprises. This Exploratory Data Analysis (EDA) utilizes data from the National Transportation Safety Board to determine which aircraft have the lowest risk. The analysis contains actionable insights for the head of the new Aviation Division, Scott Fly.

## Import Python Libraries

In [5]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Load the Data

In [6]:
!ls data

AviationData.csv  USState_Codes.csv


In [7]:
#load the CSV files for the rest of the project

#pandas says columns 6, 7, and 28 have mixed data types.  For now we will set them to strings to avoid errors later
'''
Latitude and Longitude have two formats in the file.  One is degrees, minutes, seconds format with a suffix for hemisphere like N
The other format is called decimal degrees and it is a float.
'''

#latin1 is required as utf-8 will not load
#load 5 rows just for column names, a full load shows mixed data type warnings on columns 6, 7, 28
#so we will tell pandas to load them as strings
aviation_data = pd.read_csv("data/AviationData.csv",encoding="latin1", nrows=1)
col_list = list(aviation_data.columns)
dtype_spec = {
    col_list[6]: 'str', #Latitude
    col_list[7]: 'str', #Longitude
    col_list[28]: 'str' #Broad.phase.of.flight
}

#now load it in full without warnings
aviation_data = pd.read_csv("data/AviationData.csv",encoding="latin1", dtype=dtype_spec)
uscode_data = pd.read_csv("data/USState_Codes.csv")

### Mixed Data Type Issue with Longitude, Latitude

For latitude and longitude they mix formats.  Some are in degrees, minutes and seconds format with a suffix like 'N' for direction.  Some are in decimal degrees, which are easier to work with mathematically.

In [8]:
#To know to pass str types for columns 6, 7 and 28 we had to know what is up with those columns
#We do value_counts to see if one issue comes up a lot and inspect some initial values
aviation_data['Latitude'].value_counts()

332739N      19
335219N      18
334118N      17
32.815556    17
324934N      16
             ..
039613N       1
342034N       1
433113N       1
343255N       1
373829N       1
Name: Latitude, Length: 25589, dtype: int64

In [9]:
aviation_data['Longitude'].value_counts()

0112457W       24
1114342W       18
1151140W       17
-104.673056    17
-112.0825      16
               ..
0843135W        1
0101957W        1
1064131W        1
1114414W        1
0121410W        1
Name: Longitude, Length: 27154, dtype: int64

### Mixed Data Type for Broad.phase.of.flight 
This is likely due to NaN, but needs more investigation

In [10]:
aviation_data['Broad.phase.of.flight'].head()

0      Cruise
1     Unknown
2      Cruise
3      Cruise
4    Approach
Name: Broad.phase.of.flight, dtype: object

In [11]:
aviation_data['Broad.phase.of.flight'].value_counts()

Landing        15428
Takeoff        12493
Cruise         10269
Maneuvering     8144
Approach        6546
Climb           2034
Taxi            1958
Descent         1887
Go-around       1353
Standing         945
Unknown          548
Other            119
Name: Broad.phase.of.flight, dtype: int64

In [12]:
aviation_data['Broad.phase.of.flight'].isna().sum()

27165

In [13]:
aviation_data[aviation_data['Broad.phase.of.flight'].isna()]['Broad.phase.of.flight'].head()

3030    NaN
3550    NaN
3637    NaN
4032    NaN
5505    NaN
Name: Broad.phase.of.flight, dtype: object

## Standardize Data

Lets attack getting rid of duplicate labels.  We checked ahead and there's a lot of labels duplicated for a lot of reasons.  This attempts to get rid of all of that.

In [31]:
#lowercase and stripe white space, this removes a lot of duplicate labels
def standardize_string(s):
    if pd.isna(s):
        return s
    if isinstance(s,str):
        s = s.strip().lower()#.replace(' ','_')
    return s

#for some columns treating none as nan helps us do analysis
def none_to_nan(s):
    if s == 'none':
        return np.nan
    return s

#for some columns treating unknown as nan helps us do analysis
def unknown_to_nan(s):
    if s == 'unknown':
        return np.nan
    return s

#for some columns treating unknown as nan helps us do analysis
def unk_to_nan(s):
    if s == 'unk':
        return np.nan
    return s

display(aviation_data.head())
aviation_data = aviation_data.applymap(standardize_string)

replace_none_cols = ['Airport.Code','Airport.Name','Registration.Number']
aviation_data[replace_none_cols] = aviation_data[replace_none_cols].applymap(none_to_nan)

replace_unknown_cols = ['Engine.Type','Aircraft.damage','Registration.Number']
aviation_data[replace_unknown_cols] = aviation_data[replace_unknown_cols].applymap(unknown_to_nan)

display(aviation_data.head())
aviation_data['Airport.Code'].value_counts()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218x45444,accident,sea87la080,1948-10-24,"moose creek, id",united states,NaN,NaN,NaN,NaN,...,personal,NaN,2.0,0.0,0.0,0.0,unk,cruise,probable cause,NaN
1,20001218x45447,accident,lax94la336,1962-07-19,"bridgeport, ca",united states,NaN,NaN,NaN,NaN,...,personal,NaN,4.0,0.0,0.0,0.0,unk,unknown,probable cause,19-09-1996
2,20061025x01555,accident,nyc07la005,1974-08-30,"saltville, va",united states,36.922223,-81.878056,NaN,NaN,...,personal,NaN,3.0,NaN,NaN,NaN,imc,cruise,probable cause,26-02-2007
3,20001218x45448,accident,lax96la321,1977-06-19,"eureka, ca",united states,NaN,NaN,NaN,NaN,...,personal,NaN,2.0,0.0,0.0,0.0,imc,cruise,probable cause,12-09-2000
4,20041105x01764,accident,chi79fa064,1979-08-02,"canton, oh",united states,NaN,NaN,NaN,NaN,...,personal,NaN,1.0,2.0,NaN,0.0,vmc,approach,probable cause,16-04-1980


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218x45444,accident,sea87la080,1948-10-24,"moose creek, id",united states,NaN,NaN,NaN,NaN,...,personal,NaN,2.0,0.0,0.0,0.0,unk,cruise,probable cause,NaN
1,20001218x45447,accident,lax94la336,1962-07-19,"bridgeport, ca",united states,NaN,NaN,NaN,NaN,...,personal,NaN,4.0,0.0,0.0,0.0,unk,unknown,probable cause,19-09-1996
2,20061025x01555,accident,nyc07la005,1974-08-30,"saltville, va",united states,36.922223,-81.878056,NaN,NaN,...,personal,NaN,3.0,NaN,NaN,NaN,imc,cruise,probable cause,26-02-2007
3,20001218x45448,accident,lax96la321,1977-06-19,"eureka, ca",united states,NaN,NaN,NaN,NaN,...,personal,NaN,2.0,0.0,0.0,0.0,imc,cruise,probable cause,12-09-2000
4,20041105x01764,accident,chi79fa064,1979-08-02,"canton, oh",united states,NaN,NaN,NaN,NaN,...,personal,NaN,1.0,2.0,NaN,0.0,vmc,approach,probable cause,16-04-1980


pvt     497
apa     160
ord     149
mri     137
den     115
       ... 
78oh      1
0v6       1
8c3       1
56m       1
eikh      1
Name: Airport.Code, Length: 10345, dtype: int64

## Checking for Missing Data
First we will run some checks on the aviation_data to see what we're dealing with for missing data

In [15]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            48637 non-null  object 
 9   Airport.Name            52558 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87161 non-null  object 
 14  Make                    88826 non-null

It seems we are missing a lot of data!  We will need to formulate plans on all of this.
This next code block will let us see how much data is missing percent wise

In [16]:
# Calculate the percentage of missing values for each column
missing_perc = aviation_data.isna().mean() * 100
missing_perc

Event.Id                   0.000000
Investigation.Type         0.000000
Accident.Number            0.000000
Event.Date                 0.000000
Location                   0.058500
Country                    0.254250
Latitude                  61.320298
Longitude                 61.330423
Airport.Code              45.283443
Airport.Name              40.872324
Injury.Severity            1.124999
Aircraft.damage            3.593246
Aircraft.Category         63.677170
Registration.Number        1.943998
Make                       0.070875
Model                      0.103500
Amateur.Built              0.114750
Number.of.Engines          6.844491
Engine.Type               10.268987
FAR.Description           63.974170
Schedule                  85.845268
Purpose.of.flight          6.965991
Air.carrier               81.271023
Total.Fatal.Injuries      12.826109
Total.Serious.Injuries    14.073732
Total.Minor.Injuries      13.424608
Total.Uninjured            6.650992
Weather.Condition          5

Convention suggests dropping columns where over 50% of the data is missing, unless it is very important to your analysis.
These are candidates to consider dropping:

In [17]:
missing_perc[missing_perc > 50]

Latitude             61.320298
Longitude            61.330423
Aircraft.Category    63.677170
FAR.Description      63.974170
Schedule             85.845268
Air.carrier          81.271023
dtype: float64

### Location

Lets see what Location has, maybe it will help us decide on what to do with Lat and Long

In [18]:
aviation_data['Location'].value_counts()

anchorage, ak       548
miami, fl           275
houston, tx         271
albuquerque, nm     265
chicago, il         256
                   ... 
medina, mn            1
circle pines, mn      1
pine island, fl       1
churchtown, oh        1
brasnorte,            1
Name: Location, Length: 21977, dtype: int64

We have options now, we can drop Lat and Long or we can use some sort of API to get the Lat and Long filled based on the Locatoin when it is missing!  We don't have to decide now, lets let further exploration guide our choices.

## Exploring More Columns

### Investigation.Type

In [19]:
aviation_data['Investigation.Type'].value_counts()

accident    85015
incident     3874
Name: Investigation.Type, dtype: int64

Accident - caused personal injury, damage or damage
Incident - could have potentially caused harm but didn't necessarily 

Weighting these differently is a good idea.

### Country

In [20]:
aviation_data['Country'].value_counts()

united states     82248
brazil              374
canada              359
mexico              358
united kingdom      344
                  ...  
chad                  1
ivory coast           1
cambodia              1
yemen                 1
benin                 1
Name: Country, Length: 215, dtype: int64

There is a strong bias to the United States so our data is only going to be the most reliable in the US due to uncontrolled factors like regulation differences between countries.

## Airport.Code and Airport.Name

In [21]:
aviation_data['Airport.Code'].value_counts()

pvt     497
apa     160
ord     149
mri     137
den     115
       ... 
78oh      1
0v6       1
8c3       1
56m       1
eikh      1
Name: Airport.Code, Length: 10345, dtype: int64

In [22]:
aviation_data['Airport.Name'].value_counts()

private                           471
private airstrip                  266
private strip                     161
merrill field                     109
centennial                        102
                                 ... 
lambert-st. louis int'l             1
williamson mingo cty                1
sanona creek airstrip               1
penns cave                          1
wichita dwight d eisenhower nt      1
Name: Airport.Name, Length: 21565, dtype: int64

We will have to clean the names quite a bit especially around private airports.
The codes have a lot of missing data could that correlate with private airports not having codes?

In [23]:
aviation_data[(aviation_data['Airport.Name']=='private') & (aviation_data['Airport.Code'].isna())][['Airport.Name','Airport.Code']]

,Airport.Name,Airport.Code
974,private,NaN
1057,private,NaN
1076,private,NaN
1186,private,NaN
1658,private,NaN
...,...,...
85962,private,NaN
86046,private,NaN
86494,private,NaN
88425,private,NaN


In [24]:
aviation_data[(aviation_data['Airport.Name']=='private') & (aviation_data['Airport.Code'].notna())][['Airport.Code']].value_counts()

Airport.Code
pvt             120
priv              5
rla               2
0co6              1
172               1
4nc5              1
mm20              1
my99              1
unk               1
xxx               1
dtype: int64

In [25]:
aviation_data[aviation_data['Airport.Name']=='private']['Airport.Name'].count()

471

So 337/471 of them are NaN and private.  of the remaining ones morst are pvt but priv is the same thing so we can fix that.

## Injury.Severity

In [26]:
aviation_data['Injury.Severity'].value_counts()

non-fatal     67357
fatal(1)       6167
fatal          5262
fatal(2)       3711
incident       2219
              ...  
fatal(270)        1
fatal(60)         1
fatal(43)         1
fatal(143)        1
fatal(230)        1
Name: Injury.Severity, Length: 109, dtype: int64

We will need to seperate fatal from the number to let it make more sense for our charts and analysis.

## Aircraft.damage

In [29]:
aviation_data['Aircraft.damage'].value_counts()

substantial    64148
destroyed      18623
minor           2805
unknown          119
Name: Aircraft.damage, dtype: int64

Finally a field that has little issues, just a few missing values.

## Aircraft Specs
Lets examine the group of fields that might let us identify features of the airplanes, some of the most important data

### Aircraft.Category

In [55]:
aviation_data['Aircraft.Category'].value_counts()

Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: Aircraft.Category, dtype: int64

We can get rid of all that aren't airplane, probably

### Registration.Number

In [30]:
aviation_data['Registration.Number'].value_counts()

unreg      131
unk         14
unknown     10
usaf         9
n20752       8
          ... 
n62951       1
n1013e       1
n8266r       1
n65737       1
n9026p       1
Name: Registration.Number, Length: 79093, dtype: int64

Lots of NONE might not be as useful a field.  However if some planes were just train wrecks and had lots of issues then they could be 
something worth keeping in mind when interpretting the data
### Make

In [57]:
aviation_data['Make'].value_counts()

Cessna             22227
Piper              12029
CESSNA              4922
Beech               4330
PIPER               2841
                   ...  
Leonard Walters        1
Maule Air Inc.         1
Motley Vans            1
Perlick                1
ROYSE RALPH L          1
Name: Make, Length: 8237, dtype: int64

We will need to clean this data for sure the Cessna for example is listed twice with diff capitilization
### Model

In [58]:
aviation_data['Model'].value_counts()

152              2367
172              1756
172N             1164
PA-28-140         932
150               829
                 ... 
GC-1-A              1
737-3S3             1
MBB-BK117-B2        1
GLASSAIR GL25       1
M-8 EAGLE           1
Name: Model, Length: 12318, dtype: int64

Make and Model together can form unique strings to identify types of craft so this is nice.  (It is possible for a model to have the same name and diff makes, even if rare).  This pair will be used in a lot of finding the correlations in the data.
### Amateur Built

In [59]:
aviation_data['Amateur.Built'].value_counts()

No     80312
Yes     8475
Name: Amateur.Built, dtype: int64

Since we are a business we might just remove rows about amateur built aircraft.  However maybe some of the non business made ones have properties we'd be interested in?

In [60]:
aviation_data['Number.of.Engines'].value_counts()

1.0    69582
2.0    11079
0.0     1226
3.0      483
4.0      431
8.0        3
6.0        1
Name: Number.of.Engines, dtype: int64

I'm wondering what has zero engines, gliders perhaps?  Lets find out below

In [61]:
aviation_data[aviation_data['Number.of.Engines']==0][['Engine.Type']].value_counts()

Engine.Type  
Unknown          680
None               8
NONE               2
Reciprocating      2
dtype: int64

It seems it's mostly missing data.  Also our missing data is going to need standardized to one value.

In [29]:
aviation_data['Engine.Type'].value_counts()

Reciprocating      69530
Turbo Shaft         3609
Turbo Prop          3391
Turbo Fan           2481
Unknown             2051
Turbo Jet            703
None                  19
Geared Turbofan       12
Electric              10
LR                     2
NONE                   2
Hybrid Rocket          1
UNK                    1
Name: Engine.Type, dtype: int64

### FAR.Description
FAR stands for Federal Aviation Regulations. These regulations are a comprehensive set of rules and guidelines established by the Federal Aviation Administration (FAA) to ensure the safety and efficiency of civil aviation within the United States.

Some examples:
- Part 23: Airworthiness Standards for Normal, Utility, Acrobatic, and Commuter Category Airplanes
- Part 61: Certification: Pilots, Flight Instructors, and Ground Instructors
- Part 91: General Operating and Flight Rules
- Part 121: Operating Requirements: Domestic, Flag, and Supplemental Operations

In [30]:
aviation_data['FAR.Description'].value_counts()

091                               18221
Part 91: General Aviation          6486
NUSN                               1584
NUSC                               1013
137                                1010
135                                 746
121                                 679
Part 137: Agricultural              437
UNK                                 371
Part 135: Air Taxi & Commuter       298
PUBU                                253
129                                 246
Part 121: Air Carrier               165
133                                 107
Part 129: Foreign                   100
Non-U.S., Non-Commercial             97
Non-U.S., Commercial                 93
Part 133: Rotorcraft Ext. Load       32
Unknown                              22
Public Use                           19
091K                                 14
ARMF                                  8
Part 125: 20+ Pax,6000+ lbs           5
125                                   5
107                                   4


The Data for this will need standardized as seen by the Part 91 and etc.  
on part 91 and others
https://pilotinstitute.com/part-91-vs-121-vs-135/

NUSN: Non-U.S. National (incident or accident)

"NUSN" is a code used to identify incidents or accidents involving aircraft that are not registered in the United States. This categorization helps in distinguishing between domestic and international incidents for regulatory and statistical purposes.
NUSC: Non-U.S. Commercial (incident or accident)

"NUSC" refers to incidents or accidents involving non-U.S. commercial aircraft. This code is specifically used for commercial operations, such as airlines and charter services, that are not registered in the United States.

... Understanding all the codes is going to be important if we use this field

### Schedule

In [31]:
aviation_data['Schedule'].value_counts()

NSCH    4474
UNK     4099
SCHD    4009
Name: Schedule, dtype: int64

NSCH (Non-Scheduled): Refers to flights that do not operate on a regular schedule. These can include charter flights, air taxi operations, private flights, and other ad-hoc operations.

UNK (Unknown): Indicates that the schedule type of the flight operation is unknown. This can occur when the information is not available or not recorded in the safety database.

SCHD (Scheduled): Refers to flights that operate on a regular, published schedule. These are typically commercial airline flights that follow a fixed timetable.

### Purpose.of.flight

In [32]:
aviation_data['Purpose.of.flight'].value_counts()

Personal                     49448
Instructional                10601
Unknown                       6802
Aerial Application            4712
Business                      4018
Positioning                   1646
Other Work Use                1264
Ferry                          812
Aerial Observation             794
Public Aircraft                720
Executive/corporate            553
Flight Test                    405
Skydiving                      182
External Load                  123
Public Aircraft - Federal      105
Banner Tow                     101
Air Race show                   99
Public Aircraft - Local         74
Public Aircraft - State         64
Air Race/show                   59
Glider Tow                      53
Firefighting                    40
Air Drop                        11
ASHO                             6
PUBS                             4
PUBL                             1
Name: Purpose.of.flight, dtype: int64

This is very informative as a field and it seems to have minimal issues with missing data.  Here is the Skydiving Joseph mentioned.

### Air.carrier

In [33]:
aviation_data['Air.carrier'].value_counts()

Pilot                        258
American Airlines             90
United Airlines               89
Delta Air Lines               53
SOUTHWEST AIRLINES CO         42
                            ... 
WOODY CONTRACTING INC          1
Rod Aviation LLC               1
Paul D Franzon                 1
TRAINING SERVICES INC DBA      1
MC CESSNA 210N LLC             1
Name: Air.carrier, Length: 13590, dtype: int64

It has a lot of missing rows but it could have a big impact on safety.

### Total.Fatal.Injuries

In [ ]:
aviation_data['Total.Fatal.Injuries'].value_counts()

0.0      59675
1.0       8883
2.0       5173
3.0       1589
4.0       1103
         ...  
156.0        1
68.0         1
31.0         1
115.0        1
176.0        1
Name: Total.Fatal.Injuries, Length: 125, dtype: int64

it is in floats for some reason so we might make sure there's no "partial fatal injuries"

### Total.Serious.Injuries

In [35]:
aviation_data['Total.Serious.Injuries'].value_counts()

0.0      63289
1.0       9125
2.0       2815
3.0        629
4.0        258
5.0         78
6.0         41
7.0         27
9.0         16
10.0        13
8.0         13
13.0         9
11.0         6
26.0         5
14.0         5
12.0         5
20.0         3
25.0         3
28.0         3
21.0         2
59.0         2
50.0         2
17.0         2
47.0         2
137.0        1
19.0         1
161.0        1
27.0         1
35.0         1
67.0         1
33.0         1
88.0         1
125.0        1
53.0         1
34.0         1
41.0         1
18.0         1
63.0         1
55.0         1
23.0         1
43.0         1
39.0         1
45.0         1
44.0         1
16.0         1
60.0         1
106.0        1
81.0         1
15.0         1
22.0         1
Name: Total.Serious.Injuries, dtype: int64

It is in floats again
### Total.Minor.Injuries

In [37]:
aviation_data['Total.Minor.Injuries'].value_counts()

0.0      61454
1.0      10320
2.0       3576
3.0        784
4.0        372
5.0        129
6.0         67
7.0         59
9.0         22
8.0         20
13.0        14
10.0        11
12.0        11
14.0        10
11.0         9
17.0         8
19.0         6
18.0         6
24.0         5
22.0         5
25.0         4
16.0         4
15.0         4
33.0         4
20.0         3
21.0         3
26.0         3
23.0         3
32.0         3
27.0         3
50.0         2
30.0         2
36.0         2
31.0         2
28.0         2
42.0         2
38.0         2
57.0         1
65.0         1
84.0         1
43.0         1
35.0         1
380.0        1
47.0         1
68.0         1
200.0        1
71.0         1
58.0         1
171.0        1
39.0         1
96.0         1
29.0         1
69.0         1
62.0         1
45.0         1
125.0        1
40.0         1
Name: Total.Minor.Injuries, dtype: int64

It is in floats again
### Total.Uninjured

In [38]:
aviation_data['Total.Uninjured'].value_counts()

0.0      29879
1.0      25101
2.0      15988
3.0       4313
4.0       2662
         ...  
558.0        1
412.0        1
338.0        1
401.0        1
455.0        1
Name: Total.Uninjured, Length: 379, dtype: int64

also in floats
### Weather.Condition

In [39]:
aviation_data['Weather.Condition'].value_counts()

VMC    77303
IMC     5976
UNK      856
Unk      262
Name: Weather.Condition, dtype: int64

UNK and Unk are not standardized

Conditions are codes:

**VMC (Visual Meteorological Conditions):**

VMC: Indicates weather conditions that allow for visual flight rules (VFR) operations. Pilots can navigate and control the aircraft using visual references outside the cockpit.

**IMC (Instrument Meteorological Conditions):**

IMC: Refers to weather conditions that require instrument flight rules (IFR) operations. Visibility and cloud cover are such that pilots must rely on cockpit instruments for navigation and control.

**UNK (Unknown):**

UNK: Indicates that the meteorological conditions at the time of the incident or data point are unknown or not recorded.


### Broad.phase.of.flight

In [40]:
aviation_data['Broad.phase.of.flight'].value_counts()

Landing        15428
Takeoff        12493
Cruise         10269
Maneuvering     8144
Approach        6546
Climb           2034
Taxi            1958
Descent         1887
Go-around       1353
Standing         945
Unknown          548
Other            119
Name: Broad.phase.of.flight, dtype: int64

Landing is the most prone to incident or accident it seems like, especially since it is also relatively short in terms of phases.
This data can help us find planes weak points and strong points as well as high risk areas.

### Report.Status

In [41]:
aviation_data['Report.Status'].value_counts()

Probable Cause                                                                                                                                                       61754
Foreign                                                                                                                                                               1999
<br /><br />                                                                                                                                                           167
Factual                                                                                                                                                                145
The pilot's failure to maintain directional control during the landing roll.                                                                                            56
                                                                                                                                                 

So it seems a lot of the values are probable cause

**Probable Cause**: This term is used to describe the findings of an investigation that identify the factors or events that most likely led to the incident or accident. When a report reaches the "probable cause" status, the investigating authority, such as the National Transportation Safety Board (NTSB) in the United States, has completed its analysis and has determined the primary reasons behind the occurrence.


### Publication Date

In [42]:
aviation_data['Publication.Date'].value_counts()

25-09-2020    17019
26-09-2020     1769
03-11-2020     1155
31-03-1993      452
25-11-2003      396
              ...  
29-11-2004        1
29-08-2001        1
18-11-2004        1
17-12-1996        1
29-12-2022        1
Name: Publication.Date, Length: 2924, dtype: int64

I'm not sure why most the publication dates are 25-09-2020, maybe they batch the reports or something